# Import Important Libraries

In [2]:
import re
import pickle
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dinez\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
ps = PorterStemmer()
vectorizer = TfidfVectorizer()

# Load Datasets

In [10]:
df = pd.read_csv('news data/train.csv')

In [11]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \r\nAn Iranian woman has been sentenced ...,1


# Pre-Processing

In [12]:
# Fill null values
df = df.fillna(' ')

# Feature Engineering
df['content'] = df['author']+' '+df['title']

# Stemming
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [ps.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

df['content'] = df['content'].apply(stemming)

# Split features and label
xTrain = df.drop('label',axis=1)
yTrain = df['label']

# TF-IDF Representation
xTrain = vectorizer.fit_transform(xTrain['content'])

In [13]:
print('--- Data Shape ---')
print('xTrain shape: ', xTrain.shape)
print('yTrain shape: ', yTrain.shape)

--- Data Shape ---
xTrain shape:  (20800, 17128)
yTrain shape:  (20800,)


# Model Training

In [14]:
trainX, valX, trainY, valY = train_test_split(xTrain, yTrain, test_size=0.20, random_state=42)

In [15]:
model = LogisticRegression()
model.fit(trainX, trainY)

LogisticRegression()

# Model Validation

In [16]:
predictions = model.predict(valX)

In [17]:
accuracy_score(valY, predictions)

0.9740384615384615

# Save Model

In [19]:
file = open('Fake News Classifier - LR', 'wb')
pickle.dump(model, file)

file.close()

In [20]:
# Save tf-idf vectorizer so we can use it for new data
file = open('tfidf vectorizer', 'wb')
pickle.dump(vectorizer, file)

file.close()